In [1]:
import pandas as pd
import numpy as np
from numpy import concatenate
from pandas import concat
from pandas import read_csv
from pandas import DataFrame
import tensorflow as tf
from tensorflow import keras
import os
import datetime
import IPython
import IPython.display
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import explained_variance_score, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Flatten, LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Bidirectional, Input, Reshape, RepeatVector, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
%matplotlib inline
from scipy.stats import randint as sp_randint
np.random.seed(123) # for reproducibility
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier, KerasRegressor
import keras_tuner

In [2]:
# CARGAR LA BASE DE DATOS
from pandas import read_csv
datos = read_csv('Bucaramanga_Lags.csv', encoding='latin-1', sep=";")
# Dropping the id and date columns
datos = datos.drop(['Municipio', 'Año', 'Semana_Epi','Prom_Dia_Preci_1', 'Prom_Dia_Preci_2', 'Prom_Dia_Preci_3',
       'Prom_Dia_Preci_4', 'Prom_Dia_Preci_5', 'Prom_Dia_Preci_6', 'Total_Preci_Sem_1', 'Total_Preci_Sem_2',
       'Total_Preci_Sem_3', 'Total_Preci_Sem_4', 'Total_Preci_Sem_5','Total_Preci_Sem_6'],axis=1)
datos.columns

Index(['Casos_Dengue', 'Prom_Dia_Preci', 'Total_Preci_Sem'], dtype='object')

In [3]:
train_size = int(len(datos) * 0.7)
test_size = len(datos) - train_size
train, test = datos.iloc[0:train_size], datos.iloc[train_size:len(datos)]
print(len(train), len(test))

436 188


In [4]:
f_transformer = MinMaxScaler(feature_range=(0, 1))
f_columns = ['Prom_Dia_Preci', 'Total_Preci_Sem']
f_transformer = f_transformer.fit(train[f_columns].to_numpy())
train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
train.head()

,Casos_Dengue,Prom_Dia_Preci,Total_Preci_Sem
0,24,0.000000,0.000000
1,36,0.111616,0.111562
2,31,0.109060,0.109128
3,34,0.111616,0.111562
4,33,0.000000,0.000000


In [5]:
train.describe()

,Casos_Dengue,Prom_Dia_Preci,Total_Preci_Sem
count,436.000000,436.000000,436.000000
mean,62.100917,0.105238,0.105392
std,49.327327,0.122981,0.123124
min,5.000000,0.000000,0.000000
25%,25.000000,0.019810,0.019777
50%,49.000000,0.062482,0.062475
75%,85.000000,0.141153,0.141075
max,272.000000,1.000000,1.000000


In [6]:
dengue_transformer = MinMaxScaler(feature_range=(0, 1))
dengue_transformer = dengue_transformer.fit(train[['Casos_Dengue']])
train['Casos_Dengue'] = dengue_transformer.transform(train[['Casos_Dengue']])
train['Casos_Dengue'].head()

0    0.071161
1    0.116105
2    0.097378
3    0.108614
4    0.104869
Name: Casos_Dengue, dtype: float64

In [7]:
test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())
test.head()

,Casos_Dengue,Prom_Dia_Preci,Total_Preci_Sem
436,42,0.022721,0.022718
437,53,0.173530,0.173631
438,49,0.048566,0.048682
439,34,0.028117,0.027992
440,40,0.111900,0.111968


In [8]:
test['Casos_Dengue'] = dengue_transformer.transform(test[['Casos_Dengue']])
test['Casos_Dengue'].head()

436    0.138577
437    0.179775
438    0.164794
439    0.108614
440    0.131086
Name: Casos_Dengue, dtype: float64

In [9]:
def create_dataset(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [10]:
time_steps = 10
# reshape to [samples, time_steps, n_features]
X_train, y_train = create_dataset(train, train.Casos_Dengue, time_steps)
X_test, y_test = create_dataset(test, test.Casos_Dengue, time_steps)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape )

(426, 10, 3) (426,) (178, 10, 3) (178,)


In [11]:
from tensorflow.keras.layers import InputLayer

def Dense_model(neuronas, num_layers, activation = 'relu'):
    
    mlp_model = tf.keras.Sequential()
    
    # CAPA DE ENTRADA
    mlp_model.add(tf.keras.layers.InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])))
    
    
    #CAPAS INTERMEDIAS
    for i in range(num_layers):
        mlp_model.add(tf.keras.layers.Dense(units=neuronas, activation=activation))
    
    #CAPA DE SALIDA
    mlp_model.add(tf.keras.layers.Dense(1))
    
    #COMPILACIÓN DEL MODELO
    mlp_model.compile(optimizer='adam', loss = 'mse', metrics = ['mse'])
    
    return mlp_model

In [12]:
#HIPERPARÁMETROS

neuronas = [50, 70, 100]
activation = ['sigmoid','relu', 'tanh']
#init_mode = ['uniform', 'normal', 'zero']
num_layers = [3, 5, 7]
#dropout_rate = [0.1, 0.3, 0.5]

#optimizer = ['adam', 'sgd', 'rmsprop']
#batch_size = [40, 60, 80]
#epochs = [10, 50, 100]
#momentum = [0.2, 0.4, 0.5]

param_grid = dict(model__neuronas=neuronas,
                  model__num_layers=num_layers,
                  model__activation=activation)
                  #batch_size=batch_size,
                  #epochs=epochs,
                  #optimizer__momentum=momentum
                # model__init_mode=init_mode
                #model__dropout_rate=dropout_rate,

In [13]:
model =  KerasRegressor(model=Dense_model, verbose=0)

rs = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring='neg_mean_squared_error',
                        n_jobs=-1, cv=5, n_iter=20, verbose=0,random_state = 123, return_train_score = True)

In [14]:
fitted_model = rs.fit(X_train, y_train)

In [16]:
resultados = pd.DataFrame(fitted_model.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)')\
    .drop(columns = 'params')\
    .sort_values('mean_test_score', ascending = False)\
    .head(10)

,param_model__num_layers,param_model__neuronas,param_model__activation,mean_test_score,std_test_score,mean_train_score,std_train_score
0,7,70,tanh,NaN,NaN,NaN,NaN
1,7,70,sigmoid,NaN,NaN,NaN,NaN
2,5,100,sigmoid,NaN,NaN,NaN,NaN
3,7,100,sigmoid,NaN,NaN,NaN,NaN
4,3,100,tanh,NaN,NaN,NaN,NaN
5,7,50,relu,NaN,NaN,NaN,NaN
6,3,70,tanh,NaN,NaN,NaN,NaN
7,7,50,tanh,NaN,NaN,NaN,NaN
8,3,70,relu,NaN,NaN,NaN,NaN
9,7,100,tanh,NaN,NaN,NaN,NaN


In [17]:
print("Best: %f using %s" % (fitted_model.best_score_, fitted_model.best_params_))

Best: nan using {'model__num_layers': 7, 'model__neuronas': 70, 'model__activation': 'tanh'}
